<a href="https://colab.research.google.com/github/josouane/web-crawler/blob/main/chatgpt-web-crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

class WebCrawler:
    def __init__(self, start_url, visiting_strategy='preorder'):
        self.start_url = start_url
        self.visiting_strategy = visiting_strategy.lower()
        self.visited_urls = set()
        self.corpus = {}

    def crawl(self, url):
        if url not in self.visited_urls:
            print(f"Visiting: {url}")
            self.visited_urls.add(url)
            try:
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                title = soup.title.string.strip() if soup.title else 'Untitled'
                text_content = self.extract_text_content(soup)
                self.corpus[title] = text_content

                if self.visiting_strategy == 'preorder':
                    links = self.extract_links(soup)
                    for link in links:
                        self.crawl(link)

                # Additional visiting strategies (inorder, postorder) can be implemented here

            except Exception as e:
                print(f"Error crawling {url}: {e}")

    def extract_text_content(self, soup):
        # Remove non-essential elements such as scripts, styles, etc.
        for script in soup(['script', 'style']):
            script.decompose()

        # Get the text content of the page
        text_content = soup.get_text(separator=' ', strip=True)
        return text_content

    def extract_links(self, soup):
        # Extract all links from the page
        links = [link.get('href') for link in soup.find_all('a', href=True)]
        # Filter internal links only
        links = [link for link in links if link.startswith(self.start_url)]
        return links

    def start_crawling(self):
        self.crawl(self.start_url)
        self.save_corpus()

    def save_corpus(self):
        # Save the extracted corpus to text files
        for title, content in self.corpus.items():
            with open(f"{title}.txt", 'w', encoding='utf-8') as file:
                file.write(content)

if __name__ == "__main__":
    # Get the starting URL from the user
    start_url = input("Enter the website's URL: ")

    # Instantiate the WebCrawler with the provided URL and visiting strategy
    crawler = WebCrawler(start_url=start_url, visiting_strategy='preorder')

    # Start crawling
    crawler.start_crawling()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
Visiting: https://www.stjohns.edu/
Visiting: https://www.stjohns.edu/who-we-are/faith-and-mission/campus-ministry/opportunities/plunge-program
Visiting: https://www.stjohns.edu/who-we-are/history-and-facts/vincentian-heritage
Visiting: https://www.stjohns.edu/about/leadership-and-administration/administrative-offices/human-resources/policies#notice-of-non-discrimination--equal-opportunity-
Visiting: https://www.stjohns.edu/who-we-are/campus-sustainability
Visiting: https://www.stjohns.edu/life-st-johns/career-services
Visiting: https://www.stjohns.edu/life-st-johns/career-services/leadership-development
Visiting: https://www.stjohns.edu/sites/default/files/2024-01/odk_application_-_spring_2024.pdf


Visiting: https://www.stjohns.edu/sites/default/files/2024-01/odk_benefits_of_membership_2024.pdf


Visiting: https://www.stjohns.edu/sites/default/files/2021-08/Co-Curricular%20Transcript.pdf


Visiting: https://www.stjohns.edu/uis
Visiting: https://www.stjohns.edu/about/leadership-and-administration/office-president/presidents-society
Visiting: https://www.stjohns.edu/sites/default/files/uploads/AlmaMater.pdf


Visiting: https://www.stjohns.edu/academics/programs?level%5B151%5D=151
Visiting: https://www.stjohns.edu/node/20391
Visiting: https://www.stjohns.edu/academics/programs/accounting-master-science
Visiting: https://www.stjohns.edu/admission/apply
Visiting: https://www.stjohns.edu/resources/safety/annual-security-fire-safety-report


...

Visiting: https://www.stjohns.edu/sites/default/files/2020-07/M1-12932%20MS%20Computational%20Biology%20Curriculum%20Sheet.pdf


Visiting: https://www.stjohns.edu/sites/default/files/2020-03/M1-12859%20Crim_Justice%20Curriculum%20Sheet.pdf


Visiting: https://www.stjohns.edu/sites/default/files/2020-06/M1-12907%20MS%20Enviro%20OMICS%20Curriculum%20Sheet.pdf


Visiting: https://www.stjohns.edu/sites/default/files/2019-08/M1-12539%20-%20MA%20Enviro%20Sustainability%20Curriculum%20Sheet.pdf


KeyboardInterrupt: 